<a href="https://colab.research.google.com/github/sm116327/DS/blob/main/Entrega%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Instalamos las librerías que necesitaremos a lo largo del notebook.

In [ ]:
pip install lightfm

In [ ]:
pip install python-dateutil

###Importamos librerías

In [ ]:
import numpy as np
import pandas as pd

from datetime import datetime
from scipy.sparse import csr_matrix
from lightfm import LightFM
from dateutil import tz
from dateutil.parser import isoparse
from tqdm import tqdm

### Funciones auxiliares

In [ ]:
def time_subtract(date_1, date_2):
  '''
  Description: Funciòn para sustraer dos fechas dadas en el siguiente formato -> '2021-02-18 22:52:00
  Input:
        date_1: Fecha minuendo 
        date_2: Fecha sustraendo
  Output: 
        resultado de la resta date_2 - date_1
  '''
  #format in which the time is expected
  format = '%Y-%m-%d %H:%M:%S'
  #convert input datetime to list
  val_1 = datetime.strptime(date_1, format)
  val_2 = datetime.strptime(date_2, format)
  #compute the substraction
  diff = val_2 - val_1
  #convert the value to minutesç
  #diff_min = diff.total_seconds()/60

  return diff.total_seconds() / 60

In [ ]:
def rate_calculator(time_spent, asset_time):
  '''
  Description: To calculate a rating value (interaction with a content) using the amount of time that the user spent interacting with it
  Input:
  Output:
  '''
  #threshold = 0.5

  if (time_spent / asset_time) >= 0.4:
    rate = 1
  else: 
    rate = 0

  #Binarize output using a predefined threshold value
  #if rate > threshold:
  #  rate = 1
  #else: 
  #  rate = 0

  return rate

###Importamos los datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Data Science ICARO/Proyecto 3/train (1).csv', parse_dates=['tunein', 'tuneout']) 
print(data.head())

md = pd.read_csv('/content/drive/MyDrive/Data Science ICARO/Proyecto 3/metadata.csv', sep = ';', header = 0) 
print(md.head())

   customer_id  account_id  ...             tuneout  resume
0            0       90627  ... 2021-02-18 23:35:00       0
1            0       90627  ... 2021-03-25 00:01:00       0
2            1        3387  ... 2021-03-15 10:23:00       0
3            1        3387  ... 2021-03-15 11:18:00       1
4            1        3387  ... 2021-03-16 09:44:00       0

[5 rows x 7 columns]
   asset_id  content_id  ...          start_vod_date            end_vod_date
0     15188         0.0  ...  2017-12-01T00:00:00.0Z  2020-12-01T23:59:59.0Z
1     24940         1.0  ...  2017-12-15T00:00:00.0Z  2022-12-14T23:59:59.0Z
2     21939         2.0  ...  2018-01-25T00:00:00.0Z  2020-12-01T23:59:59.0Z
3      9005         3.0  ...  2018-05-27T00:00:00.0Z  2021-04-30T23:59:59.0Z
4      7391         4.0  ...  2019-05-02T00:00:00.0Z  2020-12-31T23:59:59.0Z

[5 rows x 30 columns]


In [ ]:
data = data.dropna()

###Creamos un df temporal de prueba.

In [ ]:
data_prueba = data.head(500000)

###Creamos rating feature sobre el dataset de train usando la cantidad de tiempo que pasó el usuario mirando el contenido.

In [ ]:
%%time
#calculate the amount of time that the user spent interacting with the asset
data_prueba['time_duration_min'] = data_prueba.apply(lambda x: time_subtract(str(x['tunein']),
                                                                               str(x['tuneout'])),
                                                                                axis = 1)

CPU times: user 38.7 s, sys: 454 ms, total: 39.2 s
Wall time: 42.6 s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
%%time

#Create a new column with the asset_id duration in minutes
data_prueba['runtime_min'] = data_prueba.asset_id.apply(lambda x: md[md.asset_id == x].run_time_min.values[0])

CPU times: user 8min 9s, sys: 27.1 s, total: 8min 36s
Wall time: 8min 33s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data_prueba.sort_values("account_id", ascending=True)

,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout
782,140,8496.0,5.0,5.0,2021-01-25 23:16:00,2021-01-25 23:21:00
789,140,2709.0,2.0,91.0,2021-03-22 13:58:00,2021-03-22 14:00:00
788,140,32263.0,54.0,54.0,2021-03-21 20:42:00,2021-03-21 21:36:00
787,140,24599.0,43.0,50.0,2021-03-14 11:08:00,2021-03-14 11:51:00
786,140,26697.0,4.0,4.0,2021-01-26 23:57:00,2021-01-27 00:01:00
...,...,...,...,...,...,...
388165,113772,29263.0,34.0,169.0,2021-03-30 17:09:00,2021-03-30 17:43:00
388167,113772,7140.0,25.0,24.0,2021-03-31 01:09:00,2021-03-31 01:34:00
388168,113772,27030.0,13.0,13.0,2021-03-31 02:02:00,2021-03-31 02:15:00
388166,113772,29263.0,3.0,169.0,2021-03-30 17:43:00,2021-03-30 17:46:00


In [ ]:
#select only the needed features
columns = ['account_id', 'asset_id', 'time_duration_min', 'runtime_min', 'tunein', 'tuneout']
data_prueba = data_prueba[columns]
data_prueba.head()

,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout
0,90627,18332.0,43.0,43.0,2021-02-18 22:52:00,2021-02-18 23:35:00
1,90627,24727.0,44.0,42.0,2021-03-24 23:17:00,2021-03-25 00:01:00
2,3387,895.0,18.0,54.0,2021-03-15 10:05:00,2021-03-15 10:23:00
3,3387,895.0,55.0,54.0,2021-03-15 10:23:00,2021-03-15 11:18:00
4,3387,26062.0,20.0,49.0,2021-03-16 09:24:00,2021-03-16 09:44:00


In [ ]:
data_prueba.shape

(500000, 6)

In [ ]:
data_prueba.duplicated(subset=["account_id", "asset_id"]).value_counts()

False    363231
True     136769
dtype: int64

In [ ]:
data_prueba2 = data_prueba.groupby(["account_id", "asset_id"], as_index=False).agg({"time_duration_min":"sum", "runtime_min": "mean", "tunein":"max", "tuneout":"max"})
data_prueba2.head(40)


,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout
0,140,2709.0,2.0,91.0,2021-03-22 13:58:00,2021-03-22 14:00:00
1,140,4760.0,4.0,4.0,2021-01-26 23:53:00,2021-01-26 23:57:00
2,140,8496.0,5.0,5.0,2021-01-25 23:16:00,2021-01-25 23:21:00
3,140,9041.0,83.0,53.0,2021-01-25 21:43:00,2021-01-25 23:06:00
4,140,19842.0,5.0,5.0,2021-01-25 23:21:00,2021-01-25 23:26:00
5,140,24398.0,5.0,5.0,2021-01-26 23:48:00,2021-01-26 23:53:00
6,140,24599.0,43.0,50.0,2021-03-14 11:08:00,2021-03-14 11:51:00
7,140,26697.0,4.0,4.0,2021-01-26 23:57:00,2021-01-27 00:01:00
8,140,29866.0,5.0,5.0,2021-01-25 23:09:00,2021-01-25 23:14:00
9,140,30988.0,7.0,4.0,2021-01-25 23:06:00,2021-01-25 23:09:00


In [ ]:
# Probamos que haya sumado
data_prueba2[data_prueba2.account_id == 3387]

,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout
2410,3387,895.0,73.0,54.0,2021-03-15 10:23:00,2021-03-15 11:18:00
2411,3387,26062.0,23.0,49.0,2021-03-16 09:44:00,2021-03-16 09:47:00


In [ ]:
# Verificamos que no haya duplicados
data_prueba2[(data_prueba2.duplicated(subset= ["account_id", "asset_id"]))]

,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout


In [ ]:
data_prueba2.shape

(363231, 6)

In [ ]:
# Create a new dataset with the unique combination of 'account_id' and 'asset_id' to store the new 'rating' feature
# columns2 = ['account_id', 'asset_id', 'time_duration_min', 'runtime_min']
# df_filtered = data_prueba.drop_duplicates(subset = ['account_id', 'asset_id'])[columns2]
# df_filtered.head()

# NO UTLIZO ESTE CODIGO YA QUE EN VEZ DE ELIMINAR DUPLICADOS, LO QUE HICIMOS FUE SUMAR LOS MINUTOS QUE SE PASA VIENDO Y NO POR ELIMINAR
# PASE QUE CUANDO REALIZAMOS EL THRESHOLD NO ALCANCE EL MINIMO CUANDO EN REALIDAD LO HIZO EN DIFERENTES MOMENTOS

In [ ]:
df_filtered = data_prueba2

### Antes de traer Content ID del metadata trabajaremos con el analizando la informacion.

In [ ]:
md = md[['asset_id', 'content_id', 'show_type','run_time_min', 
         'start_vod_date', 'end_vod_date']]
md.head()

,asset_id,content_id,show_type,run_time_min,start_vod_date,end_vod_date
0,15188,0.0,Serie,48.0,2017-12-01T00:00:00.0Z,2020-12-01T23:59:59.0Z
1,24940,1.0,Película,105.0,2017-12-15T00:00:00.0Z,2022-12-14T23:59:59.0Z
2,21939,2.0,Película,82.0,2018-01-25T00:00:00.0Z,2020-12-01T23:59:59.0Z
3,9005,3.0,Película,99.0,2018-05-27T00:00:00.0Z,2021-04-30T23:59:59.0Z
4,7391,4.0,Película,126.0,2019-05-02T00:00:00.0Z,2020-12-31T23:59:59.0Z


In [ ]:
# Al ser tutoriales con poca visualizacion efectiva y parte de una misma saga, se dropean
md[pd.isnull(md.content_id)]


,asset_id,content_id,show_type,run_time_min,start_vod_date,end_vod_date
1540,15032,NaN,Tutorial,1.0,2019-07-22T00:00:00.0Z,2020-12-14T23:59:59.0Z
1541,22963,NaN,Tutorial,1.0,2019-07-22T00:00:00.0Z,2020-12-14T23:59:59.0Z
1542,32637,NaN,Tutorial,1.0,2019-07-22T00:00:00.0Z,2020-12-14T23:59:59.0Z
1543,21822,NaN,Tutorial,0.0,2019-07-22T00:00:00.0Z,2020-12-14T23:59:59.0Z
1544,17157,NaN,Tutorial,1.0,2019-07-22T00:00:00.0Z,2020-12-14T23:59:59.0Z
1545,24583,NaN,Tutorial,2.0,2019-07-22T00:00:00.0Z,2020-12-14T23:59:59.0Z
1546,27036,NaN,Tutorial,1.0,2019-07-22T00:00:00.0Z,2020-12-14T23:59:59.0Z
18197,2046,NaN,Tutorial,1.0,2019-07-22T00:00:00.0Z,2025-07-22T23:59:00.0Z
18198,1105,NaN,Tutorial,1.0,2019-07-22T00:00:00.0Z,2025-07-22T23:59:00.0Z
18199,28435,NaN,Tutorial,1.0,2019-07-22T00:00:00.0Z,2025-07-22T23:59:00.0Z


In [ ]:
df_filtered[(df_filtered.asset_id == 15188)]

,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout


In [ ]:
md = md.dropna()

In [ ]:
md.head()

,asset_id,content_id,show_type,run_time_min,start_vod_date,end_vod_date
0,15188,0.0,Serie,48.0,2017-12-01T00:00:00.0Z,2020-12-01T23:59:59.0Z
1,24940,1.0,Película,105.0,2017-12-15T00:00:00.0Z,2022-12-14T23:59:59.0Z
2,21939,2.0,Película,82.0,2018-01-25T00:00:00.0Z,2020-12-01T23:59:59.0Z
3,9005,3.0,Película,99.0,2018-05-27T00:00:00.0Z,2021-04-30T23:59:59.0Z
4,7391,4.0,Película,126.0,2019-05-02T00:00:00.0Z,2020-12-31T23:59:59.0Z


In [ ]:
#traer content_id
df_aux_md = md[['asset_id','content_id']]
df_aux_md.set_index(keys='asset_id',inplace=True)
df_aux_md.head()

,content_id
asset_id,
15188,0.0
24940,1.0
21939,2.0
9005,3.0
7391,4.0


In [ ]:
df_filtered.shape

(363231, 6)

In [ ]:
df_aux_md.shape

(33119, 1)

In [ ]:
df_filtered.head()

,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout
0,140,2709.0,2.0,91.0,2021-03-22 13:58:00,2021-03-22 14:00:00
1,140,4760.0,4.0,4.0,2021-01-26 23:53:00,2021-01-26 23:57:00
2,140,8496.0,5.0,5.0,2021-01-25 23:16:00,2021-01-25 23:21:00
3,140,9041.0,83.0,53.0,2021-01-25 21:43:00,2021-01-25 23:06:00
4,140,19842.0,5.0,5.0,2021-01-25 23:21:00,2021-01-25 23:26:00


In [ ]:
df_filtered = (df_filtered.merge(df_aux_md, how = "left", on="asset_id")).dropna()
df_filtered.head()

,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout,content_id
0,140,2709.0,2.0,91.0,2021-03-22 13:58:00,2021-03-22 14:00:00,550.0
1,140,4760.0,4.0,4.0,2021-01-26 23:53:00,2021-01-26 23:57:00,3276.0
2,140,8496.0,5.0,5.0,2021-01-25 23:16:00,2021-01-25 23:21:00,3276.0
3,140,9041.0,83.0,53.0,2021-01-25 21:43:00,2021-01-25 23:06:00,2120.0
4,140,19842.0,5.0,5.0,2021-01-25 23:21:00,2021-01-25 23:26:00,3276.0


In [ ]:
df_filtered.shape

(363216, 7)

In [ ]:
df_filtered.isna().sum()

account_id           0
asset_id             0
time_duration_min    0
runtime_min          0
tunein               0
tuneout              0
content_id           0
dtype: int64

In [ ]:
df_filtered[df_filtered.duplicated(subset=['account_id','content_id'])]

,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout,content_id
2,140,8496.0,5.0,5.0,2021-01-25 23:16:00,2021-01-25 23:21:00,3276.0
4,140,19842.0,5.0,5.0,2021-01-25 23:21:00,2021-01-25 23:26:00,3276.0
5,140,24398.0,5.0,5.0,2021-01-26 23:48:00,2021-01-26 23:53:00,3276.0
7,140,26697.0,4.0,4.0,2021-01-26 23:57:00,2021-01-27 00:01:00,3276.0
8,140,29866.0,5.0,5.0,2021-01-25 23:09:00,2021-01-25 23:14:00,3276.0
...,...,...,...,...,...,...,...
363221,113764,7265.0,68.0,57.0,2021-03-29 01:24:00,2021-03-29 02:32:00,2043.0
363223,113764,12329.0,46.0,56.0,2021-03-28 21:40:00,2021-03-28 22:26:00,2043.0
363224,113764,17429.0,10.0,52.0,2021-03-31 14:09:00,2021-03-31 14:19:00,2120.0
363225,113764,20145.0,63.0,53.0,2021-03-28 23:31:00,2021-03-29 00:34:00,2043.0


In [ ]:
df_filtered = df_filtered.drop_duplicates(subset=['account_id','content_id'],keep='first')

In [ ]:
df_filtered[(df_filtered["time_duration_min"].values/df_filtered["runtime_min"].values)>=0.2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout,content_id
1,140,4760.0,4.0,4.0,2021-01-26 23:53:00,2021-01-26 23:57:00,3276.0
3,140,9041.0,83.0,53.0,2021-01-25 21:43:00,2021-01-25 23:06:00,2120.0
6,140,24599.0,43.0,50.0,2021-03-14 11:08:00,2021-03-14 11:51:00,2200.0
11,141,22838.0,62.0,78.0,2021-03-23 00:30:00,2021-03-23 01:32:00,4334.0
12,142,3031.0,90.0,161.0,2021-02-26 23:29:00,2021-02-26 23:47:00,2570.0
...,...,...,...,...,...,...,...
363219,113764,3740.0,50.0,51.0,2021-03-29 00:34:00,2021-03-29 01:24:00,2043.0
363222,113764,9041.0,77.0,53.0,2021-03-30 14:30:00,2021-03-30 15:47:00,2120.0
363227,113772,7140.0,25.0,24.0,2021-03-31 01:09:00,2021-03-31 01:34:00,135.0
363228,113772,27030.0,13.0,13.0,2021-03-31 02:02:00,2021-03-31 02:15:00,670.0


In [ ]:
df_filtered["ratings"] = df_filtered["time_duration_min"].values/df_filtered["runtime_min"].values
df_filtered

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.


,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout,content_id,ratings
0,140,2709.0,2.0,91.0,2021-03-22 13:58:00,2021-03-22 14:00:00,550.0,0.021978
1,140,4760.0,4.0,4.0,2021-01-26 23:53:00,2021-01-26 23:57:00,3276.0,1.000000
3,140,9041.0,83.0,53.0,2021-01-25 21:43:00,2021-01-25 23:06:00,2120.0,1.566038
6,140,24599.0,43.0,50.0,2021-03-14 11:08:00,2021-03-14 11:51:00,2200.0,0.860000
11,141,22838.0,62.0,78.0,2021-03-23 00:30:00,2021-03-23 01:32:00,4334.0,0.794872
...,...,...,...,...,...,...,...,...
363222,113764,9041.0,77.0,53.0,2021-03-30 14:30:00,2021-03-30 15:47:00,2120.0,1.452830
363227,113772,7140.0,25.0,24.0,2021-03-31 01:09:00,2021-03-31 01:34:00,135.0,1.041667
363228,113772,27030.0,13.0,13.0,2021-03-31 02:02:00,2021-03-31 02:15:00,670.0,1.000000
363229,113772,29263.0,37.0,169.0,2021-03-30 17:43:00,2021-03-30 17:46:00,524.0,0.218935


In [ ]:
# Definimos un resultado binario donde 1 represente que supera nuestro threshold y 0 no. 
# El threshold es de 0,4 entre el cociente de lo que cada cuenta vio en cada activo sobre el tiempo que dura ese asset id
df_filtered["ratings"] = np.where(df_filtered['ratings'] >= 0.4, 1, 0)
df_filtered.head()

,account_id,asset_id,time_duration_min,runtime_min,tunein,tuneout,content_id,ratings
0,140,2709.0,2.0,91.0,2021-03-22 13:58:00,2021-03-22 14:00:00,550.0,0
1,140,4760.0,4.0,4.0,2021-01-26 23:53:00,2021-01-26 23:57:00,3276.0,1
3,140,9041.0,83.0,53.0,2021-01-25 21:43:00,2021-01-25 23:06:00,2120.0,1
6,140,24599.0,43.0,50.0,2021-03-14 11:08:00,2021-03-14 11:51:00,2200.0,1
11,141,22838.0,62.0,78.0,2021-03-23 00:30:00,2021-03-23 01:32:00,4334.0,1


In [ ]:
df_filtered.shape

(129604, 8)

In [ ]:
df_filtered = df_filtered.groupby(["account_id", "content_id", "tunein", "tuneout", "ratings", "runtime_min"], as_index=False, dropna=True).agg({"time_duration_min": "sum"})

In [ ]:
df_filtered.head()


,account_id,content_id,tunein,tuneout,ratings,runtime_min,time_duration_min
0,140,550.0,2021-03-22 13:58:00,2021-03-22 14:00:00,0,91.0,2.0
1,140,2120.0,2021-01-25 21:43:00,2021-01-25 23:06:00,1,53.0,83.0
2,140,2200.0,2021-03-14 11:08:00,2021-03-14 11:51:00,1,50.0,43.0
3,140,3276.0,2021-01-26 23:53:00,2021-01-26 23:57:00,1,4.0,4.0
4,141,4334.0,2021-03-23 00:30:00,2021-03-23 01:32:00,1,78.0,62.0


In [ ]:
%%time

#unique user_id values within the dataset
# user_ids = df_filtered.account_id.value_counts().index

#for every unique user_id value
#for user_id in user_ids:
  #bring all the unique assets (contents) that the user interact with
# asset_ids = df_filtered[df_filtered.account_id == user_id].asset_id.value_counts().index
  #for every unique asset
#  for asset_id in asset_ids:
    #create a temporal dataframe
#    temp_val = data_prueba[(data_prueba.asset_id == asset_id) & (data_prueba.account_id == user_id)]
    #total amount of time the user spent watching this unique asset
#    time_duration_min = np.sum(temp_val.time_duration_min.values)
    #calculate the rating using the time spent and the total duration of the asset
#    rating = rate_calculator(time_duration_min,
#                             temp_val.runtime_min.values[0])
    #create a mask (indexes) and assign the new rating values to the filtered dataset
#    mask = df_filtered[(df_filtered.asset_id == asset_id) & (df_filtered.account_id == user_id)].index
#    df_filtered.loc[mask, 'rating'] = rating 

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.48 µs


###Separamos en train y test el dataset df_filtered

In [ ]:
print(df_filtered['tunein'].min())
print(df_filtered['tunein'].max())

2021-01-01 00:00:00
2021-03-31 23:59:00


In [ ]:
train = df_filtered[(df_filtered['tunein'] >= datetime(year= 2021, month= 1, day=1)) &
              (df_filtered['tunein'] < datetime(year=2021, month=3, day=1))]
train.head()

,account_id,content_id,tunein,tuneout,ratings,runtime_min,time_duration_min
1,140,2120.0,2021-01-25 21:43:00,2021-01-25 23:06:00,1,53.0,83.0
3,140,3276.0,2021-01-26 23:53:00,2021-01-26 23:57:00,1,4.0,4.0
5,142,525.0,2021-01-22 21:51:00,2021-01-22 23:03:00,1,74.0,72.0
6,142,1529.0,2021-02-16 12:13:00,2021-02-16 12:21:00,1,7.0,8.0
7,142,1709.0,2021-01-21 23:42:00,2021-01-21 23:56:00,0,98.0,14.0


In [ ]:
#amount of rows and columns in train dataset
print(train.shape)

#amount of different account_id
print(train.account_id.nunique())

(85416, 7)
15596


In [ ]:
test = df_filtered[(df_filtered['tunein'] >= datetime(year = 2021, month= 3, day =1))]
test.head()

,account_id,content_id,tunein,tuneout,ratings,runtime_min,time_duration_min
0,140,550.0,2021-03-22 13:58:00,2021-03-22 14:00:00,0,91.0,2.0
2,140,2200.0,2021-03-14 11:08:00,2021-03-14 11:51:00,1,50.0,43.0
4,141,4334.0,2021-03-23 00:30:00,2021-03-23 01:32:00,1,78.0,62.0
8,142,1876.0,2021-03-06 09:57:00,2021-03-06 10:09:00,0,120.0,19.0
9,142,2524.0,2021-03-11 16:54:00,2021-03-11 18:16:00,1,82.0,164.0


In [ ]:
#amount of rows and columns in test dataset
print(test.shape)

#amount of different account_id 
print(test.account_id.nunique())

(44188, 7)
12750


A continuación, vemos si tenemos usuarios cold start.

In [ ]:
test[~test.account_id.isin(train.account_id.unique())].account_id.nunique() 

2338

Dropeamos las columnas 'tunein' y 'tuneout' de nuestros datasets ya que no los necesitaremos.

In [ ]:
columns = ['account_id', 'content_id', 'ratings']
df_filtered = df_filtered[columns]
train = train[columns]
test = test[columns]

###Creamos matriz de interacciones.

In [ ]:
matrix_col =  ['account_id','content_id','ratings']
interact = df_filtered[matrix_col]
interact[interact["account_id"]== 3388]

,account_id,content_id,ratings
871,3388,691.0,1
872,3388,2040.0,0
873,3388,2100.0,1
874,3388,3038.0,1
875,3388,3386.0,1
876,3388,3487.0,0


In [ ]:
inter_matrix = pd.pivot_table(interact, index="account_id", columns="content_id", values="ratings")
inter_matrix.head()

content_id,0.0,1.0,3.0,6.0,7.0,8.0,9.0,18.0,19.0,20.0,22.0,23.0,24.0,26.0,33.0,36.0,37.0,38.0,39.0,40.0,43.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,...,4329.0,4330.0,4331.0,4332.0,4333.0,4334.0,4335.0,4336.0,4337.0,4338.0,4339.0,4340.0,4341.0,4342.0,4343.0,4344.0,4345.0,4346.0,4347.0,4348.0,4349.0,4350.0,4351.0,4352.0,4353.0,4354.0,4355.0,4356.0,4357.0,4358.0,4359.0,4360.0,4361.0,4362.0,4363.0,4364.0,4365.0,4366.0,4368.0,4369.0
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
140,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
141,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
143,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
144,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
inter_matrix.fillna(0, inplace = True)
inter_matrix.head()

content_id,0.0,1.0,3.0,6.0,7.0,8.0,9.0,18.0,19.0,20.0,22.0,23.0,24.0,26.0,33.0,36.0,37.0,38.0,39.0,40.0,43.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,...,4329.0,4330.0,4331.0,4332.0,4333.0,4334.0,4335.0,4336.0,4337.0,4338.0,4339.0,4340.0,4341.0,4342.0,4343.0,4344.0,4345.0,4346.0,4347.0,4348.0,4349.0,4350.0,4351.0,4352.0,4353.0,4354.0,4355.0,4356.0,4357.0,4358.0,4359.0,4360.0,4361.0,4362.0,4363.0,4364.0,4365.0,4366.0,4368.0,4369.0
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
inter_matrix = inter_matrix.drop(columns=inter_matrix.columns[0])

In [ ]:
user_id = list(inter_matrix.index)
user_dict = {}
counter = 0 
for i in user_id:
    user_dict[i] = counter
    counter += 1

In [ ]:
user_dict

{140: 0,
 141: 1,
 142: 2,
 143: 3,
 144: 4,
 145: 5,
 146: 6,
 147: 7,
 148: 8,
 149: 9,
 150: 10,
 151: 11,
 152: 12,
 153: 13,
 154: 14,
 155: 15,
 156: 16,
 157: 17,
 158: 18,
 159: 19,
 160: 20,
 161: 21,
 162: 22,
 163: 23,
 164: 24,
 165: 25,
 166: 26,
 167: 27,
 168: 28,
 169: 29,
 170: 30,
 171: 31,
 172: 32,
 173: 33,
 174: 34,
 175: 35,
 176: 36,
 177: 37,
 178: 38,
 179: 39,
 180: 40,
 181: 41,
 182: 42,
 183: 43,
 184: 44,
 185: 45,
 186: 46,
 187: 47,
 188: 48,
 189: 49,
 190: 50,
 191: 51,
 192: 52,
 193: 53,
 194: 54,
 195: 55,
 196: 56,
 197: 57,
 198: 58,
 199: 59,
 200: 60,
 201: 61,
 202: 62,
 203: 63,
 204: 64,
 205: 65,
 206: 66,
 207: 67,
 208: 68,
 209: 69,
 210: 70,
 211: 71,
 212: 72,
 213: 73,
 214: 74,
 215: 75,
 216: 76,
 217: 77,
 218: 78,
 219: 79,
 220: 80,
 221: 81,
 222: 82,
 223: 83,
 224: 84,
 225: 85,
 226: 86,
 227: 87,
 228: 88,
 229: 89,
 230: 90,
 231: 91,
 232: 92,
 233: 93,
 234: 94,
 235: 95,
 236: 96,
 237: 97,
 238: 98,
 239: 99,
 240: 100,

In [ ]:
item_id = list(inter_matrix.columns)
item_dict = {}
counter = 0 
for i in item_id:
    item_dict[i] = counter
    counter += 1

In [ ]:
interactions_matrix_csr = csr_matrix(inter_matrix.values)
interactions_matrix_csr

<17934x3752 sparse matrix of type '<class 'numpy.float64'>'
	with 74597 stored elements in Compressed Sparse Row format>

In [ ]:
model = LightFM(random_state=0,
                loss='warp',
                learning_rate=0.03,
                no_components=100)

model = model.fit(interactions_matrix_csr,
                  epochs=100,
                  num_threads=16, verbose=False)

In [ ]:
md.end_vod_date = [isoparse(x) for x in md.end_vod_date]

In [ ]:
content_stale = md[md.end_vod_date >= datetime(year=2021, month=3, day=31, tzinfo=tz.gettz('UTC'))]
content_stale

,asset_id,content_id,show_type,run_time_min,start_vod_date,end_vod_date
1,24940,1.0,Película,105.0,2017-12-15T00:00:00.0Z,2022-12-14 23:59:59+00:00
3,9005,3.0,Película,99.0,2018-05-27T00:00:00.0Z,2021-04-30 23:59:59+00:00
5,17760,5.0,TV,44.0,2019-06-27T00:00:00.0Z,2021-07-27 23:59:59+00:00
10,21684,9.0,Película,132.0,2020-01-03T00:00:00.0Z,2021-12-31 23:59:59+00:00
142,12502,36.0,Película,93.0,2020-08-30T00:00:00.0Z,2021-05-31 23:59:00+00:00
...,...,...,...,...,...,...
33137,19624,4371.0,Película,92.0,2021-05-06T00:00:00.0Z,2023-05-06 23:59:59+00:00
33138,21935,4260.0,Película,77.0,2021-03-16T00:00:00.0Z,2023-03-16 23:59:00+00:00
33139,3683,1979.0,Serie,64.0,2021-03-07T00:00:00.0Z,2021-11-20 23:59:00+00:00
33140,16810,1979.0,Serie,56.0,2021-02-21T00:00:00.0Z,2021-11-20 23:59:00+00:00


In [ ]:
stale_content = md[md.end_vod_date <= datetime(year=2021, month=3, day=31, tzinfo=tz.gettz('UTC'))]
stale_content

,asset_id,content_id,show_type,run_time_min,start_vod_date,end_vod_date
0,15188,0.0,Serie,48.0,2017-12-01T00:00:00.0Z,2020-12-01 23:59:59+00:00
2,21939,2.0,Película,82.0,2018-01-25T00:00:00.0Z,2020-12-01 23:59:59+00:00
4,7391,4.0,Película,126.0,2019-05-02T00:00:00.0Z,2020-12-31 23:59:59+00:00
6,3619,6.0,Serie,59.0,2019-11-26T00:00:00.0Z,2020-12-01 23:59:00+00:00
7,2472,7.0,TV,182.0,2019-12-02T00:00:00.0Z,2020-12-01 23:59:59+00:00
...,...,...,...,...,...,...
33128,19105,1216.0,Serie,45.0,2021-01-23T00:00:00.0Z,2021-02-23 23:59:00+00:00
33133,15082,1836.0,Serie,45.0,2021-03-02T00:00:00.0Z,2021-03-04 23:59:00+00:00
33134,31035,1205.0,Rolling,43.0,2021-01-20T00:00:00.0Z,2021-01-27 23:59:00+00:00
33141,20468,943.0,TV,24.0,2021-02-17T00:00:00.0Z,2021-03-19 23:59:59+00:00


In [ ]:
# Prueba si el content 2200 estan en aquellos que estan en stale content, es decir ese content id vence antes del 31/03
stale_content[(stale_content.content_id == 2200)]

,asset_id,content_id,show_type,run_time_min,start_vod_date,end_vod_date


In [ ]:
df_filtered.head()

,account_id,content_id,ratings
0,140,550.0,0
1,140,2120.0,1
2,140,2200.0,1
3,140,3276.0,1
4,141,4334.0,1


In [ ]:
popularity_df =df_filtered.groupby("content_id", as_index = False).agg({"ratings":"sum"}).sort_values(by="ratings", ascending=False)
popularity_df

,content_id,ratings
1617,2040.0,2019
3335,3900.0,1245
3545,4133.0,916
3244,3806.0,820
1562,1983.0,627
...,...,...
3205,3766.0,0
1239,1633.0,0
1251,1647.0,0
1259,1655.0,0


In [ ]:
#popular content
#popularity_df =df_filtered.groupby("content_id", as_index = False).agg({"account_id":"nunique"}).sort_values(by="account_id", ascending=False)

popularity_df.columns=["content_id", "popularity"]
#popularity_df.head()


In [ ]:
popularity_df.shape

(3753, 2)

In [ ]:
# El content valido va a ser aquel que encuentre en stale_content ya que este tiene como condicion aquellos que finalizan antes del 31/03 
# Con lo cual correriamos el riesgo de que le recomendemos una pelicula que ya no estaria disponible.
popular_content = [x for x in popularity_df.content_id.values if x not in stale_content.content_id][:20]

popular_content

[3900.0,
 729.0,
 3384.0,
 3382.0,
 712.0,
 3572.0,
 774.0,
 3386.0,
 3578.0,
 3863.0,
 3906.0,
 3592.0,
 3402.0,
 3897.0,
 3530.0,
 3902.0,
 3389.0,
 184.0,
 3397.0,
 3343.0]

In [ ]:
# Ejemplo opuesto al anterior ya que si no hicieramos este filtro, las recomendaciones estarian variando y tendriamos
# el riesgo dicho anteriormente de que el contenido ya no este disponible
popular_content_des = [x for x in popularity_df.content_id.values if x not in content_stale.content_id][:20]

popular_content_des

[2040.0,
 4133.0,
 3806.0,
 1983.0,
 2160.0,
 116.0,
 1316.0,
 1877.0,
 1573.0,
 1139.0,
 2942.0,
 3716.0,
 3210.0,
 1985.0,
 2627.0,
 604.0,
 1020.0,
 1462.0,
 3726.0,
 3381.0]

In [ ]:
%%time
model = model.fit(interactions_matrix_csr, epochs=10)

CPU times: user 5.28 s, sys: 7.68 ms, total: 5.28 s
Wall time: 5.39 s


In [ ]:
user_x = user_dict[140]
n_users, n_items = inter_matrix.shape
item_ids = np.arange(n_items)
#ACA GENERAMOS PREDICCIONES
preds = model.predict(user_ids=user_x, item_ids = item_ids)

In [ ]:
scores = pd.Series(preds) # 

In [ ]:
scores.index = inter_matrix.columns
recomms_140 = ([x for x in scores.sort_values(ascending=False).index if x not in stale_content.content_id])[:20]
recomms_140

[3900.0,
 729.0,
 3384.0,
 3402.0,
 802.0,
 3863.0,
 184.0,
 327.0,
 3382.0,
 3393.0,
 3592.0,
 3868.0,
 817.0,
 3897.0,
 3906.0,
 3578.0,
 3844.0,
 3569.0,
 3929.0,
 3480.0]

In [ ]:
# scores = pd.Series(preds)
# scores.index= inter_matrix.columns
# recomms_140 = list(pd.Series(scores.sort_values(ascending=False).index))[:20] #obtenemos las primeras 20 


In [ ]:
# recomms_140

In [ ]:
inter_matrix

content_id,1.0,3.0,6.0,7.0,8.0,9.0,18.0,19.0,20.0,22.0,23.0,24.0,26.0,33.0,36.0,37.0,38.0,39.0,40.0,43.0,45.0,46.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,66.0,...,4329.0,4330.0,4331.0,4332.0,4333.0,4334.0,4335.0,4336.0,4337.0,4338.0,4339.0,4340.0,4341.0,4342.0,4343.0,4344.0,4345.0,4346.0,4347.0,4348.0,4349.0,4350.0,4351.0,4352.0,4353.0,4354.0,4355.0,4356.0,4357.0,4358.0,4359.0,4360.0,4361.0,4362.0,4363.0,4364.0,4365.0,4366.0,4368.0,4369.0
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
142,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
143,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113743,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
113764,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
pd.Series(test.account_id.unique())

0           140
1           141
2           142
3           145
4           146
          ...  
12745    113733
12746    113743
12747    113764
12748    113772
12749    113789
Length: 12750, dtype: int64

In [ ]:
#definimos dict donde vamos a ir almacenando las recomendaciones
recomms_dict = {
    'user_id': [],
    'recomms': []
}

#obtenemos cantidad de usuarios y cantidad de items
n_users, n_items = inter_matrix.shape
item_ids = np.arange(n_items)

#por cada usuario del dataset de test, generamos recomendaciones
for user in tqdm(test.account_id.unique()):
    #COMPLETAR: Validar si el usuario se encuentra en la matriz de interacciones (inter_matrix.index)
    if user in list(inter_matrix.index):
      # Si el usuario esta en train, no es cold start. Usamos el modelo para recomendar
      user_x = user_dict[user] #buscamos el indice del usuario en la matriz (transformamos id a indice)

      #COMPLETAR: Generar las predicciones para el usuario x
      preds = model.predict(user_ids=user_x, item_ids = item_ids)

      #COMPLETAR: Basándose en el ejemplo anterior, ordenar las predicciones de menor a mayor y quedarse con 50.
      scores = pd.Series(preds)
      scores.index = inter_matrix.columns
      scores = list(pd.Series(scores.sort_values(ascending=False).index))[:20]

      #COMPLETAR: Obtener listado de contenidos vistos anteriormente por el usuario (en el set de train)
      watched_contents = df_filtered[df_filtered.account_id == user].content_id.unique()

      #COMPLETAR: Filtrar contenidos ya vistos y quedarse con los primeros 10
      recomms = [x for x in scores if (x not in watched_contents) & (x not in stale_content)] [:20]

      # Guardamos las recomendaciones en el diccionario
      recomms_dict['user_id'].append(user)
      recomms_dict['recomms'].append(recomms)
    
    # En este else trataremos a los usuarios que no están en la matriz (cold start)
    else:
      recomms_dict['user_id'].append(user)
      # Les recomendamos contenido popular
      recomms_dict['recomms'].append(popular_content)

100%|██████████| 12750/12750 [02:20<00:00, 91.03it/s]


In [ ]:
recomms_df = pd.DataFrame(recomms_dict)
recomms_df[2:3]

,user_id,recomms
2,142,"[3572.0, 2040.0, 36.0, 1139.0, 1005.0, 680.0, ..."


###Evaluación con MAP

Primero ordenamos los contenidos según los ratings (mayor a menor) que dieron los usuarios en el set de test.

Luego, agrupamos y armamos un listado de libros para cada usuario.

Este es el listado "ideal" contra el que vamos a comparar nuestras recomendaciones.

In [ ]:
test.head()

,account_id,content_id,ratings
0,140,550.0,0
2,140,2200.0,1
4,141,4334.0,1
8,142,1876.0,0
9,142,2524.0,1


In [ ]:

test['rbinar'] = np.where(test['ratings'] == 1, True, False)
test.head()

,account_id,content_id,ratings,rbinar
0,140,550.0,0,False
2,140,2200.0,1,True
4,141,4334.0,1,True
8,142,1876.0,0,False
9,142,2524.0,1,True


In [ ]:
test.sort_values([ "ratings"], ascending=False)

,account_id,content_id,ratings,rbinar
92478,36358,3600.0,1,True
53149,31124,591.0,1,True
84135,35166,116.0,1,True
109602,38619,244.0,1,True
109603,38619,262.0,1,True
...,...,...,...,...
62666,32327,3381.0,0,False
62664,32327,3035.0,0,False
62663,32327,2167.0,0,False
62662,32327,281.0,0,False


In [ ]:
ideal_recomms = test[test.rbinar]\
                  .sort_values(by=["account_id", "rbinar"], ascending=False)\
                  .groupby(["account_id"], as_index=False)\
                  .agg({"content_id": "unique"})
ideal_recomms.head()

,account_id,content_id
0,140,[2200.0]
1,141,[4334.0]
2,142,"[2524.0, 3189.0, 3210.0, 4102.0]"
3,146,[4249.0]
4,148,"[1946.0, 1983.0, 2174.0, 3386.0, 4133.0]"


In [ ]:
recomms_df.head()

,user_id,recomms
0,140,"[3806.0, 3900.0, 2040.0, 4133.0, 3091.0, 1983...."
1,141,"[3900.0, 4133.0, 3806.0, 2040.0, 3502.0, 3598...."
2,142,"[3572.0, 2040.0, 36.0, 1139.0, 1005.0, 680.0, ..."
3,145,"[2040.0, 3900.0, 3806.0, 4133.0, 1983.0, 2160...."
4,146,"[2040.0, 3900.0, 1316.0, 1139.0, 2160.0, 4133...."


In [ ]:
df_map = ideal_recomms.merge(recomms_df, how="left", left_on="account_id", right_on = 'user_id')[["account_id", "content_id", "recomms"]]
df_map.columns = ["account_id", "ideal", "recomms"]
df_map.head()

,account_id,ideal,recomms
0,140,[2200.0],"[3806.0, 3900.0, 2040.0, 4133.0, 3091.0, 1983...."
1,141,[4334.0],"[3900.0, 4133.0, 3806.0, 2040.0, 3502.0, 3598...."
2,142,"[2524.0, 3189.0, 3210.0, 4102.0]","[3572.0, 2040.0, 36.0, 1139.0, 1005.0, 680.0, ..."
3,146,[4249.0],"[2040.0, 3900.0, 1316.0, 1139.0, 2160.0, 4133...."
4,148,"[1946.0, 1983.0, 2174.0, 3386.0, 4133.0]","[3900.0, 3806.0, 2040.0, 3381.0, 3384.0, 1005...."


In [ ]:
aps = [] # lista vacía para ir almacenando la AP de cada recomendación

for pred, label in df_map[["recomms", "ideal"]].values:
  n = len(pred) # cantidad de elementos recomendados
  arange = np.arange(n, dtype=np.int32) + 1. # indexamos en base 1 
  rel_k = np.in1d(pred[:n], label) # lista de booleanos que indican la relevancia de cada ítem
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum() # lista con el contador de verdaderos positivos
  denom = arange[rel_k] # posiciones donde se encuentran los ítems relantes
  ap = (tp / denom).sum() / len(label) # average precision
  aps.append(ap)

In [ ]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.0
